In [121]:
#import libraries
from bs4 import BeautifulSoup as bs
import pandas as pd 
import numpy as np
import requests
from splinter import Browser
import codecs
import pymongo
#set path to use splinter
executable_path = {'executable_path': 'chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

#declare url and parser
url = "https://en.wikipedia.org/wiki/Lists_of_shipwrecks"
browser.visit(url)
html = browser.html
soup = bs(html,'html.parser')

url = []
#find desired data
for i in soup.find('ul').findAll('a'):
    url.append(i['href'])


    #click link to go to country
   # browser.click_link_by_partial_href(i['href'])
   # browser.back()

    #obtain new page html and parse
    #html_country = browser.html
    #soup_country = bs(html_country,"html.parser")

    #find all tables
    #wikitables = soup.findAll("table", class_ ='wikitable').findall()
    #pd.read_html("https://en.wikipedia.org" + i["href"])
    #if i > 1:
    #     break
tables = []
for i in url:
    tables.append(pd.read_html("https://en.wikipedia.org" + i))

#shipwreck_df = pd.DataFrame(tables)

# df = tables[0][0]
# df.columns = ['ship','sunk_date','notes','coordinates']
# print(df.head())

df = tables[0][2]
#df.columns = ['ship','sunk_date','notes','coordinates']
#print(df.head())
#df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df
print(df)

unpacklist = [x for table in tables for x in table]
new_unpacklist = unpacklist.remove(unpacklist[0])
new_df = pd.concat(unpacklist)
#new_df = pd.DataFrame(unpacklist)
#new_df.columns = ['ship','sunk_date','notes','coordinates']
#new_df= df.reindex(df.index.drop(0)).reset_index(drop=True)

ship_wreck_data = new_df.drop(columns = [4,5,6,7,8])
ship_wreck_data = ship_wreck_data.dropna()
ship_wreck_data.columns= ['ship','sunk_date','notes','coordinates']
#ship_wreck_data = df.reindex(df.index.drop(0)).reset_index(drop=True)


ship_wreck_data = ship_wreck_data.drop_duplicates()
ship_wreck_data = ship_wreck_data.drop([0])


             0              1  \
0         Ship      Sunk date   
1   Globe Star  27 April 1973   
2  HMS Gulland  13 April 1951   
3   MV Mtongwe  27 April 1994   

                                                   2  \
0                                              Notes   
1      A cargo ship that ran aground off Mombasa.[1]   
2  A 545-ton Isles-class trawler built for World ...   
3  A Likoni and Mombasa route ferry that capsized...   

                                                3  
0                                     Coordinates  
1       4°04′54″S 39°43′12″E﻿ / ﻿4.0818°S 39.72°E  
2  04°02′50″S 39°43′57″E﻿ / ﻿4.04722°S 39.73250°E  
3                                             NaN  


In [122]:
ship_wreck_data = ship_wreck_data.reset_index(drop=True)
#test = ship_wreck_data['coordinates'].str.encode("utf-8")
#test  = test.str.decode("utf-8")
#test 

#ship_wreck_data['coordinates'] = test

In [123]:
true_coordinates = []
#ship_wreck_data1 = ship_wreck_data.drop([0,74])

for coordinate in ship_wreck_data['coordinates']:
    if coordinate.find('/')==-1:
        print(coordinate)
    else:
        latlong = coordinate.split('/')[1]
        new_latlong = latlong.replace("°","").strip()
        lat = new_latlong.split()[0]
        long = new_latlong.split()[1]
        lat = lat[1:]
    
        #print(lat[:len(lat)-2])
        if lat[len(lat)-1] == 'S':
            lat = 0-float(lat[:len(lat)-2])
           # print(lat)
        else:
            lat = lat.strip("N")
            #print(lat)
        if long[len(long)-1] == 'W':
            long = 0-float(long[:len(long)-2])
            #print(long)
        else:
            long = long.strip("E")
        #print(long)
    clean_coordinates = [lat,long]
    print(clean_coordinates)
    true_coordinates.append(clean_coordinates)

[-4.081, '39.72']
[-4.0472, '39.73250']
[-25.58, '32.983']
[-13.04, '45.195']
['2', '47']
['9.533', '50.983']
[-7.8683, '39.24000']
[-6.1483, '39.19333']
[-5.6563, '39.47417']
[-1.9838, '32.31861']
[-0.6563, '32.14472']
[-17.0025, '49.84194']
['36.750', '5.183']
['36.733', '1.517']
['36.867', '5.017']
['37.000', '3.000']
['36.82028', '3.16528']
['36.250', '0.250']
['37.02000', '5.20167']
['36.93333', '5.66667']
['31.300', '29.817']
['31.750', '25.517']
['31.46', '28.00']
['32.07333', '24.07833']
['32.07583', '23.97250']
['32.06', '24.46']
['32.09778', '24.00000']
['32.983', '22.533']
['31.917', '25.183']
['33.03', '22.04']
['35.500', -6.28]
['33.667', -7.58]
['35.983', -5.28]
['33.667', -7.58]
['34.267', -6.68]
['33.667', -7.58]
['35.767', -6.03]
['35.900', -5.75]
['35.900', -5.86]
['36.87167', '11.14000']
['36.8833', '11.3167']
['36.83333', '11.16667']
['37.38333', '9.86667']
['37.317', '9.933']
[-18.1, '11.55']
[-23.9952, '14.45722']
[-25.73233, '14.833389']
[-26.37, '15.04028']
[-22

In [159]:
#convert coordinates into dataframe to clean data
true_coordinates_df = pd.DataFrame(true_coordinates)
true_coordinates_df.columns = ['Latitude', 'Longitude']

#drop duplicate values
true_coordinates_df =true_coordinates_df.drop_duplicates()

#convert true_coordinates_df to a list then into a series so both values will be in one column as coordinates
coordinate = true_coordinates_df.values.tolist()
coordinate_series = pd.Series(coordinate)

#append coordinate_series to coordinate column and drop Nan values
ship_wreck_df = ship_wreck_data
ship_wreck_df['coordinates'] = coordinate_series
ship_wreck_df =ship_wreck_df.dropna()
ship_wreck_df

,ship,sunk_date,notes,coordinates
0,Globe Star,27 April 1973,A cargo ship that ran aground off Mombasa.[1],"[-4.081, 39.72]"
1,HMS Gulland,13 April 1951,A 545-ton Isles-class trawler built for World ...,"[-4.0472, 39.73250]"
2,Katina P,26 April 1992,A Greek oil tanker that was damaged in a storm...,"[-25.58, 32.983]"
3,Sunny South,20 February 1861,An American-built extreme clipper sold to Hava...,"[-13.04, 45.195]"
4,MS Achille Lauro,2 December 1994,A cruise ship that was hijacked in 1985 by the...,"[2, 47]"
5,U-852,3 May 1944,A Type IXD2 U-boat that was attacked by Britis...,"[9.533, 50.983]"
6,SMS Königsberg,11 July 1915,A Königsberg-class light cruiser that was sunk...,"[-7.8683, 39.24000]"
7,HMS Pegasus,20 September 1914,A Pelorus-class protected cruiser that was sun...,"[-6.1483, 39.19333]"
8,MV Spice Islander I,10 September 2011,A RORO ferry that sank between Unguja and Pemb...,"[-5.6563, 39.47417]"
9,MV Bukoba,21 May 1996,A Lake Victoria ferry that sank off Mwanza wit...,"[-1.9838, 32.31861]"


In [186]:
#convert dataframe to json
ship_wreck_json = ship_wreck_df.to_json(orient='records')
ship_wreck_json

'[{"ship":"Globe Star","sunk_date":"27 April 1973","notes":"A cargo ship that ran aground off Mombasa.[1]","coordinates":[-4.081,"39.72"]},{"ship":"HMS Gulland","sunk_date":"13 April 1951","notes":"A 545-ton Isles-class trawler built for World War II. It ran aground three miles (4.8\\u00a0km) north of Mombasa.","coordinates":[-4.0472,"39.73250"]},{"ship":"Katina P","sunk_date":"26 April 1992","notes":"A Greek oil tanker that was damaged in a storm, spilling several thousand tonnes of oil.","coordinates":[-25.58,"32.983"]},{"ship":"Sunny South","sunk_date":"20 February 1861","notes":"An American-built extreme clipper sold to Havana and put to work in the slave trade. It was captured by the Royal Navy and used as a store ship, before striking a reef and sinking at Mayotte.","coordinates":[-13.04,"45.195"]},{"ship":"MS Achille Lauro","sunk_date":"2 December 1994","notes":"A cruise ship that was hijacked in 1985 by the Palestine Liberation Front. It ultimately sank after an explosion in th

In [183]:
import pymongo
#create mongo database
myclient = pymongo.MongoClient('mongodb://localhost:27017/')

db = myclient['ship_wreck_db']

In [185]:
json_data = db['ship_wreck_data']
json_data.insert_many(ship_wreck_json)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping